In [ ]:
from ufl.core.expr import Expr
from ufl import Form, inner, grad, TrialFunction, TestFunction, dx, ds
from dolfinx.fem import FunctionSpace

from lucifex.mesh import create_rectangle
from lucifex.fem import StaticFunction, StaticConstant
from lucifex.solver import bvp_solver, BoundaryConditions
from lucifex.viz import plot_colormap


def poisson(
    u: StaticFunction,
    f: StaticFunction | StaticConstant | Expr,
) -> tuple[Form, Form]:
    """
    `∇²u = f`
    """
    v = TrialFunction(u.function_space)
    u = TestFunction(u.function_space)
    F_lhs = -inner(grad(v), grad(u)) * dx
    F_rhs = v * f * dx


    g, c = ...
    F_neumann = v * g
    F_robin = v * (u - c)
    return F_lhs, -F_rhs


Lx = 2.0
Ly = 1.0
mesh = create_rectangle(Lx, Ly, 10, 10)
boundary = {
        "left": lambda x: x[0],
        "right": lambda x: x[0] - Lx,
        "bottom": lambda x: x[1],
        "top": lambda x: x[1] - Ly,
    }


fs = FunctionSpace(mesh, ('P', 1))
f = StaticFunction(fs)
f.interpolate(lambda x: x[0] * x[1] **2)
u = StaticFunction(fs)

pbc_x = BoundaryConditions(
        ("periodic", boundary["left"], boundary["right"]),
    )
pbc_xy = BoundaryConditions(
                ("periodic", boundary["left"], boundary["right"]),
                ("periodic", boundary["bottom"], boundary["top"]),
            )
u_solver = bvp_solver(poisson, ...)(u, f)
u_solver.solve()
fig, ax = plot_colormap(u)